![variamos_logo](./images/variamos_logo.png)

# 1. JSON Representation of Feature Models

`Preamble Here`

## Table of Contents

<a href='#feature_models'>1.1 Feature Models</a>  
<a href='#JSON_feature_models'>1.2 JSON Representation of Feature Models</a>  
<a href='#JSON_schema_feature_models'>1.3 JSON Schema for Validating Feature Models</a>  

In [4]:
from jsonschema import validate

## 1.1 Feature Models <a id='feture_models'></a>  

A feature model (FM) defines the valid combination of features in a product line (a domain of related products). “A FM consists of one or more feature diagrams plus additional information” [24] such as feature descriptions, additional constraints, binding times, priorities, stakeholders, etc. A feature diagram is a tree-like structure in which nodes represent features, and edges illustrate the relationships among them. A feature is defined as a prominent or distinctive user-visible aspect, quality, or characteristic of a software system [25]. Figure 1 shows a simplified example of afeature model representing an extract of a plug-in system.

Feature models were first introduced in 1990 as a part of the FODA (Feature-Oriented Domain Analysis) method [25] as a means to represent the commonalities and variabilities of product line system. Since then, feature modeling has been widely adopted by the software product line community and a number of extensions have been proposed in attempts to improve and enrich feature models. Two of these extensions are: introduction of cardinalities [24, 26] and use of attributes in feature models [27, 28, 29]. There is no consensus on a notation to define attributes, but most proposals agree that an attribute is a variable with a name, a domain and a value. A formal semantic of feature models is presented by Schobbens et al. [15] by means of the Free feature diagram language.

**Relationships in FMs can be summarized as follow:**

| Relationship | Description |
| --- | --- |
| Mandatory | If the father feature is selected then the child feature must be selected also (and vice versa). |
| Optional | If the father feature is selected then the child feature can be but must not be selected. However, if the child feature is selected then the father feature must also be selected. |
| Requires | This is a relationship between two different features (F1 requires F2). If F1 is selected, F2 has to be selected as well, but not vice-versa. It means that F2 can be selected when F1 is not selected. |
| Exclusion | This is a relationship between two different features. It indicates that both features cannot be selected together in one product. |
| Group cardinality | A group cardinality is an interval denoted &lt;n..m&gt;, with n as lower bound and m as upper bound limiting the number of child features that can be part of a product when its parent feature is selected. If one of the child features is selected, the father feature must be   elected as well. A group cardinality aggregates the optional relationships between the father feature and two or more child features and not the features itself. |

**Running Example:** Consider the hypothetical case of a plug-in product line, represented by means of a feature model in Figure 1. To construct one plug-in, users must to select 2 or 3 kinds of capabilities; configuration wizard and configuration of resolution are optional features. There are three kinds of capabilities: transformation, verification and analysis. Besides, users can dispose of one of two kinds of screen resolutions. The first case corresponds to a standard resolution (RStandard) where users have the option to select one value among {800, 1024, 1280}. If this kind of resolution is selected, the value of the Width feature must be equal to the value of
RStandard and the value of the Height feature must be equal to the RStandard multiplied by ¾. For example, if RStandard is equal to 800, then Width must be equal to 800 and Height to 600. In the second case, if R1024x768 is selected, then Width must be equal to 1024 and Height to 768. To finish, if Resolution is not selected, then the values of both Width and Height must be 0 (the default).  

<img src="./images/feature_model_example.png" width="800" height="800" align="center"/>

## 1.2 JSON Representation of Feature Models <a id='JSON_feature_models'></a>  

Feature model have been represented in different ways: XML, 3-CNF, mxGraph, are just a few examples on how feature models have been serialized. It is important for the sake of formally representing the Feature Models to adopt a DSL so that models can be easily leaded and exported for analysis. 

A great feature of having a standard representation for FMs is that you can convert in between format by using the DSL as intermediate language.

The representation whe've built follows a simple data structure that correlates the feature model with its features and then each feature with its restrictions, as shown in the graph below.


![feature_data_model](./images/feature_data_model.png)

In [40]:
feature_model = {
    'name': 'Plugin Feature Model',
    'author': 'Francisco J. Piedrahita',
    'description': '''feature model in Figure 1. To construct one plug-in, users 
     must to select 2 or 3 kinds of capabilities; configuration wizard and configuration of 
     resolution are optional features. There are three kinds of capabilities: transformation, 
     verification and analysis.''',
    'features': [
        {
            'id': 1,
            'name': 'Plugin',
            'constraints': [
                {
                    'feature_type': 'root',
                },
                {
                    'feature_type': 'mandatory',
                    'destination': [4]
                },
                {
                    'feature_type': 'optional',
                    'destination': [2, 3]
                },
            ],
        },
        {
            'id': 2,
            'name': 'Resolution',
            'constraints': [                
                {
                    'feature_type': 'optional',
                    'destination': [5, 6]
                },
                {
                    'feature_type': 'requires',
                    'destination': [3]
                }
            ],            
        },
        {
            'id': 3,
            'name': 'Configuration Wizard',
            'constraints': [
                {
                    'feature_type': 'mandatory',
                    'destination': [7, 8]
                },                
            ],            
        },
        {
            'id': 4,
            'name': 'Capabilities',
            'constraints': [                
                {
                    'feature_type': 'group_cardinality',
                    'low_threshold': 2,
                    'high_threshold': 3,
                    'destination': [9, 10, 11]
                },                
            ],            
        },
        {
            'id': 5,
            'name': 'Resolution 1024x768',            
            'constraints': [
                {
                    'feature_type': 'excludes',
                    'destination': [6]
                },                
            ],   
        },
        {
            'id': 6,
            'name': 'Resolution Standard',
            'constraints': [
                {
                    'feature_type': 'excludes',
                    'destination': [5]
                },                
            ],   
        },
        {
            'id': 7,
            'name': 'Height',
        },
        {
            'id': 8,
            'name': 'Width',
        },
        {
            'id': 9,
            'name': 'Analysis',
        },
        {
            'id': 10,
            'name': 'Verification',
        },
        {
            'id': 11,
            'name': 'Transformation',     
        },
    ]
}

## 1.3 JSON Schema for Validating Feature Models <a id='JSON_schema_feature_models'></a>  

For us it is important that whatever DSL we use it would allow us to simply the verification process of the output Feature Models. Ensuring structural and semantic consistency between models is key to achieve a consistent evaluation of the Feature Models.

JSON Schema is a great tool for reaching those goals, it will give us the tools to validate structure.

In [1]:
feature_model_schema = {
    'type': 'object',
    'properties': {
        'name': {
            'type' : 'string'
        },
        'author': {
            'type' : 'string'
        },
        'description': {
            'type' : 'string'
        },
        'features': {
            'type' : 'array',
            'items': {
                'type': 'object',
                'properties': {
                    'id': {
                        'type': 'integer'
                    },
                    'name': {
                        'type': 'string'
                    },
                    'constraints': {
                        'type': 'array',
                        'items': {
                            'allOf': [
                                {
                                    'if': {
                                        'properties': {
                                            'feature_type': { 
                                                'const': 'root',
                                            }
                                        }
                                    },
                                    'then': {
                                        'properties': {
                                            'feature_type': { 
                                                'type': 'string',
                                            }
                                        },
                                        'additionalProperties': False,
                                    }
                                },
                                
                                {
                                    'if': {
                                        'properties': {
                                            'feature_type': {
                                                'enum': ['mandatory', 'optional', 'excludes']
                                            }
                                        }
                                    },
                                    'then': {
                                        'properties': {
                                            'feature_type': { 
                                                'type': 'string',
                                            },
                                            'destination': {
                                                'type': 'array',
                                                'items': {
                                                    'type': 'integer'
                                                }
                                            }
                                        },
                                        'additionalProperties': False,  
                                    }
                                },
                                
                                {
                                    'if': {
                                        'feature_type': {
                                            'type': {
                                                'const': 'group_cardinality',
                                            }
                                        }
                                    },
                                    'then': {
                                        'properties': {
                                            'feature_type': { 
                                                'type': 'string',
                                            },
                                            'low_threshold': {
                                                'type': 'integer',
                                            },
                                            'high_threshold': {
                                                'type': 'integer',
                                            },
                                            'destination': {
                                                'type': 'array',
                                                'items': {
                                                    'type': 'integer'
                                                }
                                            }
                                        },
                                        'additionalProperties': False,
                                    }
                                }
                            ]
                        }
                    }
                },
                'required': ['id'],
            },
        },
    },
    'required': ['features'],
}

In [44]:
validate(instance=feature_model, schema=feature_model_schema)